In [8]:
import ipytest
from aoc import submit
import re

DAY = 18
ipytest.autoconfig()

In [9]:
rx_explode_pair = re.compile(r"\[(\d+),(\d+)]")
rx_explode_ls = re.compile(r"(\d+)(?!.*\d+)")
rx_explode_rs = re.compile(r"\d+")
rx_d = re.compile(r"\d+")


def replace_match(string, replacement, match, offset=0):
    span = match.span()
    return string[:span[0] + offset] + replacement + string[span[1] + offset:]


def find_explode_index(snailfish):
    count = 0
    for match in re.finditer("[\[\]]", snailfish):
        count += 1 if match.group() == '[' else -1
        if count == 5:
            return max(0, match.span()[0])


def explode(snailfish):
    index = find_explode_index(snailfish)
    if index is not None:
        pair = rx_explode_pair.search(snailfish[index:])
        ls = rx_explode_ls.search(snailfish[:index])
        rs = rx_explode_rs.search(snailfish[index + pair.end():])
        x, y = pair.groups()
        if rs:
            snailfish = replace_match(snailfish, str(int(y) + int(rs.group())), rs, index + pair.end())
        snailfish = replace_match(snailfish, '0', pair, index)
        if ls:
            snailfish = replace_match(snailfish, str(int(x) + int(ls.group())), ls)
        return snailfish, True
    return snailfish, False


def split(snailfish):
    match = re.search(r"\d{2,}", snailfish)
    if match:
        value = int(match.group())
        return replace_match(snailfish, f"[{value // 2},{value // 2 + (value % 2)}]", match), True
    return snailfish, False


def add(snail, fish):
    return f"[{snail},{fish}]"


def reduce(snailfish):
    reduced = True
    while reduced:
        snailfish, reduced = explode(snailfish)
        if not reduced:
            snailfish, reduced = split(snailfish)
    return snailfish


def magnitude(snailfish):
    return eval(snailfish.replace("[", "(3*").replace(",", "+2*").replace("]", ")"))


In [10]:
@submit(day=DAY, skip_input=False)
def part_one(raw):
    snailfish, *rest = raw.splitlines()
    for other in rest:
        snailfish = reduce(add(snailfish, other))
    return magnitude(snailfish)

part_one:
✅ example: 4140           (37.35 ms)
✅ input:   4116           (237.26 ms)


In [11]:
@submit(day=DAY)
def part_two(raw):
    from itertools import product
    result = 0
    for snail, fish in product(raw.splitlines(), repeat=2):
        if snail != fish:
            result = max(result, magnitude(reduce(add(snail, fish))))
    return result


part_two:
✅ example: 3993           (47.06 ms)
✅ input:   4638           (3989.57 ms)


In [12]:
%%ipytest -qq --color=no

import pytest


@pytest.mark.parametrize('snailfish,expected', [
    ("[[[[[9,8],1],2],3],4]", "[[[[0,9],2],3],4]"),
    ("[1,[[[[9,8],1],2],3],4]", "[10,[[[0,9],2],3],4]"),
    ("[7,[6,[5,[4,[3,2]]]]]", "[7,[6,[5,[7,0]]]]"),
    ("[[6,[5,[4,[3,2]]]],1]", "[[6,[5,[7,0]]],3]"),
    ("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]", "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"),
    ("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]", "[[3,[2,[8,0]]],[9,[5,[7,0]]]]"),
    ("[[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]",
     '[[[[5,0],[[9,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]'),
    ("[[[[5,0],[[9,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]",
     "[[[[5,9],[0,[16,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]")
])
def test_explode(snailfish, expected):
    reduced, matched = explode(snailfish)
    assert reduced == expected


def test_add():
    assert add("[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]") == "[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]"


def test_magnitude():
    assert magnitude("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]") == 3488


..........                                                                                   [100%]
